# E. faecium nanopore workflow
This notebook details the full computational workflow for the paper: "Unmasking a Superbug: Nanopore Sequencing Reveals Chromosomal *vanB* and Resistance Plasmids in VRE *Enterococcus faecium* ST177".

It is a "hybrid" workflow, meaning it contains:
1.  **Local steps** you can run directly in this notebook.
2.  **HPC steps** that require transferring files to a cluster.

## Before You Begin
This notebook **will not work** unless you complete these three setup steps first.

### Get the Data
input your Nanopore data in the <code>data/raw/</code> folder

### Create the Environment
This notebook is best run using the <code>efaecium_env</code> conda environment.

In [ ]:
#-------------[ SET UP CONDA ENVIRONMENT ]-------------

!conda env create -f "../environment.yml"
# This creates the efaecium_env conda environment


Activate this environment for this Jupyter kernel before running the cells below

### Configure Your Paths
Edit the <code>config.yaml</code> file in the project's root directory. All local file paths in this notebook are loaded from that single config file.

The HPC paths and settings are loaded from the <code>HPC_config.py</code> file located in the <code>HPC/</code> folder



## Imports and Configuration

In [ ]:
#-------------[ IMPORTS ]-------------
import yaml
import os

In [ ]:
#-------------[ CONFIG ]-------------
# Get the path to the config file
config_path = os.path.join('..', 'config.yaml')

# Load configurations from the yaml file
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
print("Configuration loaded successfully.")

# Get the raw file path
raw_fastq_path = os.path.join('..',config['raw_fastq_path'])
print(f"the raw fastq path is: {raw_fastq_path}")

# Get the Fastplong output path and proccesed file path
fastplong_output_path = os.path.join('..', config['fastplong_output_path'])
fastplong_html_path = os.path.join(fastplong_output_path, 'fastplong_report.html')
fastplong_json_path = os.path.join(fastplong_output_path, 'fastplong_report.json')
print(f"The Fastplong output path is: {fastplong_output_path}")
fastplong_filtered_path = os.path.join('..', config['fastplong_filtered_path'])
print(f"The Fastplong filtered path is: {fastplong_filtered_path}")

# Get the MultiQC input and output path
multiqc_input_path = os.path.join('..', config['multiqc_input_path'])
print(f"The MultiQC input path is: {multiqc_input_path}")
multiqc_output_path = os.path.join('..', config['multiqc_output_path'])
print(f"The MultiQC output path is: {multiqc_output_path}")

# Get the Bakta database path and input genome path
bakta_db_folder = os.path.join('..', config['bakta_db_path'])
bakta_db_path = os.path.join(bakta_db_folder, 'db')
print(f"The Bakta database path is: {bakta_db_path}")
bakta_input_path = os.path.join('..', config['flye_assembly_path'])
print(f"The bakta_input_path path is: {bakta_input_path}")
bakta_output_path = os.path.join('..', config['bakta_output_path'])
print(f"The Bakta output path is: {bakta_output_path}")

# Get the AMRFinderPlus output path
amrfinder_output_path = os.path.join('..', config['amrfinder_output_file'])
print(f"The AMRFinderPlus output path is: {amrfinder_output_path}")

# Get the MLST output path
mlst_output_file = os.path.join('..', config['mlst_output_file'])

## Initial processing and quality control

In [ ]:
#-------------[ FASTPLONG ANALYSIS ]-------------
# make output directory if it doesn't exist
!mkdir -p {fastplong_output_path}

# Run Fastplong to filter and generate reports
!fastplong \
  -i {raw_fastq_path} \
  -o {fastplong_filtered_path} \
  -h {fastplong_html_path} \
  -j {fastplong_json_path}

In [ ]:
#-------------[ INITIAL MULTIQC ]-------------

!multiqc {multiqc_input_path} \
    --title "Initial QC" \
    --filename "initial_QC" \
    --outdir {multiqc_output_path} \
    --dirs --dirs-depth 2 --force

Let's move over to the cluster to run Kraken2, Flye, and QUAST

## High Performance Cluster: Contmination Check and Assembly 
Certain resource intensive tasks or those that take huge databases are better run on a HPC

Here we will run Kraken2, Flye, and QUAST on a cluster using the supplied singularity containers. 

You will need to have downloaded a Kraken2 database to your HPC for that step to work, beware that this file is very large.


1. **Configuration**: Edit the <code>HPC_config.py</code> script to match your cluster's file paths.
2. **Transfer files**: Use scp to transfer the whole <code>scripts/HPC</code> folder as well as the input fasta file from <code>data/processed</code>
3. **Run jobs**: Log into the HPC and execute the following in the terminal from the folder you uploaded:
    1.  Run contamination check and assembly<code>python 00_HPC_orchestrator --kraken2 --flye</code>
    2.  Once flye is done, run <code>python 00_HPC_orchestrator --quast</code>
5. **Download Results**: Return to your local environment and use scp to download the entire <code>results/</code> directory from the cluster.

## Annotation


In [ ]:
#-------------[ MULTIQC REPORT POST HPC  ]-------------
!multiqc {multiqc_input_path} \
    --title "Post_HPC_QC" \
    --filename "Post_HPC_QC" \
    --outdir {multiqc_output_path} \
    --dirs --dirs-depth 2 --force

In [ ]:
# Before running bakta, download the database
# The light database is ~4GB, the full database is ~83GB
# If you have the time ans space change --type light to --type full
!bakta_db download --output {bakta_db_folder} --type light

Bacta didn't like being run from a jupyter script, just run this on the terminal:
<code>bakta --db data/bakta_database/db.full/ --output results/bakta_output/ data/processed/assembly.fasta --force</code>

In [ ]:
#-------------[ MULTIQC REPORT WITH BAKTA ]-------------
!multiqc {multiqc_input_path} \
    --title "Bakta_QC" \
    --filename "Bakta_QC" \
    --outdir {multiqc_output_path} \
    --dirs --dirs-depth 2 --force

After running bakta, run AMRFinderPlus to reveal AMR genes

In [ ]:
# First download the AMRFinderPlus database into your evnironment
!amrfinder --update 

In [ ]:
#-------------[ RUN AMRFinderPlus SEPARATELY ]-------------#
# Run amrfinder
!amrfinder  -n {bakta_input_path}\
            -O Enterococcus_faecium\
            -o {amrfinder_output_path}\

In [ ]:
#-------------[ RUN MLST TO GET SEQUENCE TYPE ]-------------
!mlst {bakta_input_path} > {mlst_output_file}

## Analysis of results

**Identifying Plasmids using PlasmidFinder 2.1**

Upload assembly file from flye to https://cge.food.dtu.dk/services/PlasmidFinder/ and download the results file

**Genomic Feature Analysis Using Gemeni 2.5**

The .tsv output of bakta was run through Gemini, a Large Language Model (LLM) available at https://gemini.google.com/app 

The model was used as an analytical tool to rapidly synthesize the large dataset and prompted to identify complex gene clusters. All identified features, operon structures, and gene groupings suggested by the LLM were subsequently manually verified against the primary Bakta annotations and cross-referenced with relevant literature.

This analysis revealed many interesting genomic features, indluding a cluster of AMR genes that AMRFinderPlus missed on one of the plasmids I thought unimportant, highlighting the need for many-pronged analysis when characterizing complex genomes.